# SENTIMENT ANALYSIS WITH SPARK ML

# Spark ML Main Concepts

- **DataFrame**: a table with built-in operations

- **Transformer**: transforms one DataFrame into another DataFrame

- **Estimator**: eg. a learning algorithm that trains on a DataFrame and produces a Model

- **Pipeline**: chains Transformers and Estimators to produce a Model

- **Evaluator**: measures how well a fitted Model does on held-out test data

# Amazon product data
We will use a [dataset](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz)[1] that contains 8.9M book reviews from Amazon, spanning May 1996 - July 2014.

Dataset characteristics:
- Number of reviews: 8.9M
- Size: 8.8GB (uncompressed)
- HDFS blocks: 70 (each with 3 replicas)


[1] Image-based recommendations on styles and substitutes
J. McAuley, C. Targett, J. Shi, A. van den Hengel
SIGIR, 2015
http://jmcauley.ucsd.edu/data/amazon/

# Load Data

In [1]:
%%time
raw_reviews = sqlContext.read.json('data/amazon/reviews_Books_5.json')

CPU times: user 8.26 ms, sys: 1.2 ms, total: 9.46 ms
Wall time: 37.8 s


In [3]:
%%time
all_reviews = raw_reviews.select('reviewText', 'overall')
all_reviews.cache()
all_reviews.show(2)

+--------------------+-------+
|          reviewText|overall|
+--------------------+-------+
|Spiritually and m...|    5.0|
|This is one my mu...|    5.0|
+--------------------+-------+
only showing top 2 rows

CPU times: user 3.88 ms, sys: 998 µs, total: 4.88 ms
Wall time: 3.01 s


# Prepare data
We will avoid neutral reviews by keeping only reviews with 1 or 5 stars overall score.
We will also filter out the reviews that contain no text.

In [5]:
nonneutral_reviews = all_reviews.filter(
    (all_reviews.overall == 1.0) | (all_reviews.overall == 5.0))
reviews = nonneutral_reviews.filter(all_reviews.reviewText != '')

In [6]:
reviews.cache()
all_reviews.unpersist()

DataFrame[reviewText: string, overall: double]

# Split Data

In [8]:
trainingData, testData = reviews.randomSplit([0.8, 0.2])

# Generate Pipeline
![pipeline](http://hadoop.cesga.es/files/sentiment_analysis/pipeline.jpg)

## Binarizer
A transformer to convert numerical features to binary (0/1) features

In [9]:
from pyspark.ml.feature import Binarizer

binarizer = Binarizer(threshold=2.5, inputCol='overall', outputCol='label')

## Tokenizer
A transformer that converts the input string to lowercase and then splits it by white spaces.

In [11]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")

## StopWordsRemover
A transformer that filters out stop words from input. Note: null values from input array are preserved unless adding null to stopWords explicitly.

In [13]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="filtered")

## HashingTF
A Transformer that converts a sequence of words into a fixed-length feature Vector. It maps a sequence of terms to their term frequencies using a hashing function.

In [1]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol="features")

NameError: name 'remover' is not defined

# Estimator
## LogisticRegression

In [17]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Pipeline

In [18]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[binarizer, tokenizer, remover, hashingTF, lr])

In [19]:
%%time
pipeLineModel = pipeline.fit(trainingData)

CPU times: user 16 ms, sys: 2.71 ms, total: 18.7 ms
Wall time: 38.7 s


# Evaluation

In [20]:
%%time
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()

predictions = pipeLineModel.transform(testData)

aur = evaluator.evaluate(predictions)

print 'Area under ROC: ', aur

/usr/hdp/2.4.2.0-258/spark/python/pyspark/ml/classification.py:207: UserWarning: weights is deprecated. Use coefficients instead.
  warnings.warn("weights is deprecated. Use coefficients instead.")


Area under ROC:  0.966076886439
CPU times: user 2.34 s, sys: 1.34 s, total: 3.68 s
Wall time: 19.9 s


# Hyperparameter Tuning

In [21]:
%%time
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
param_grid = ParamGridBuilder() \
            .addGrid(hashingTF.numFeatures, [10000, 100000]) \
            .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
            .addGrid(lr.maxIter, [10, 20]) \
            .build()
            
cv = (CrossValidator()
      .setEstimator(pipeline)
      .setEvaluator(evaluator)
      .setEstimatorParamMaps(param_grid)
      .setNumFolds(3))

cv_model = cv.fit(trainingData)

CPU times: user 20.2 s, sys: 10.8 s, total: 31.1 s
Wall time: 12min 46s


In [22]:
%%time
new_predictions = cv_model.transform(testData)
new_aur = evaluator.evaluate(new_predictions)
print 'Area under ROC: ', new_aur

Area under ROC:  0.968664347321
CPU times: user 942 ms, sys: 561 ms, total: 1.5 s
Wall time: 12.6 s
